In [1]:
import pandas as pd
import numpy as np

In [46]:
vec_file = '/Users/aravindhnivas/Documents/ML-properties/[PHYSICAL CONSTANTS OF ORGANIC COMPOUNDS]/tmp_C_processed_data/analysis_data/filtered/tmpC_topelements_processed_data/embedded_vectors/VICGAE_embeddings.npy'
csv_file = '/Users/aravindhnivas/Documents/ML-properties/[PHYSICAL CONSTANTS OF ORGANIC COMPOUNDS]/tmp_C_processed_data/analysis_data/filtered/tmpC_topelements.csv'

vectors = np.load(vec_file)
vectors = np.array(vectors)
print(vectors.shape)

df = pd.read_csv(csv_file)
df = df.set_index('INDEX')
# df = pd.read_csv(csv_file)
df.tail(15)

(7476, 32)


,CRC Index,SMILES,tmp/ºC,Processed tmp/ºC
INDEX,,,,
7625,10871,O=C=Nc1ccc(-c2ccccc2)cc1,56,56.0
7626,10872,Cc1cccc(OCC(O)CNC(C)(C)C)c1C,57,57.0
7627,10876,Cc1cccc(O)c1C,72.7(3),72.7
7628,10877,Cc1ccc(O)c(C)c1,25(1),25.0
7629,10878,Cc1ccc(C)c(O)c1,74.9(2),74.9
7630,10879,Cc1cccc(C)c1O,45.4(4),45.4
7631,10880,Cc1ccc(O)cc1C,65.1(2),65.1
7632,10881,Cc1cc(C)cc(O)c1,63.4(3),63.4
7633,10882,Cc1cc(C2(c3cc(C)c(O)c(CN(CC(=O)O)CC(=O)O)c3)OS...,286 dec,286.0


In [103]:
smi_df = df['SMILES']
smi_df.to_csv('./smiles.csv')

In [97]:
invalid_indices_full = ['# Invalid']
invalid_indices_full.extend([7637, 7639])
invalid_indices_full.extend(['# Invalid'])
invalid_indices_full.extend([7639, 7638])
np.savetxt('./invalid_indices_full.txt', invalid_indices_full, fmt='%s')

In [98]:
val = np.loadtxt('./invalid_indices_full.txt')

# unique values
invalid_ind = np.unique(val)
print(invalid_ind)
valid_df = df.drop(invalid_ind)
valid_df

[7637. 7638. 7639.]


,CRC Index,SMILES,tmp/ºC,Processed tmp/ºC
INDEX,,,,
0,0,COP(=S)(OC)Oc1ccc(Sc2ccc(OP(=S)(OC)OC)cc2)cc1,31.6(5),31.6
1,1,CC(C)C1=CC2=CCC3C(C)(C(=O)O)CCCC3(C)C2CC1,173.5,173.5
2,2,CC(C=CC1(O)C(C)=CC(=O)CC1(C)C)=CC(=O)O,160,160.0
3,3,COc1ccc(-c2cc(=O)c3c(O)cc(O)cc3o2)cc1,263,263.0
4,4,CCCC(=O)Nc1ccc(OCC(O)CNC(C)C)c(C(C)=O)c1,121,121.0
...,...,...,...,...
7632,10881,Cc1cc(C)cc(O)c1,63.4(3),63.4
7633,10882,Cc1cc(C2(c3cc(C)c(O)c(CN(CC(=O)O)CC(=O)O)c3)OS...,286 dec,286.0
7634,10884,OC1O[C@H](CO[C@@H]2OC[C@@H](O)[C@H](O)[C@H]2O)...,210,210.0


In [73]:
# invalid vec are where all elements are 0
invalid_vec_ind = np.all(vectors == 0, axis=1)
invalid_vec = vectors[invalid_vec_ind]
invalid_vec_ind.sum()

276

In [37]:
invalid_vec_ind, invalid_vec

(array([False, False, False, ...,  True, False, False]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]))

In [51]:
invalid_smiles = df['SMILES'].loc[invalid_vec_ind]
invalid_smiles.to_csv('invalid_smiles.csv')

In [38]:
from typing import Union


def convert_to_float(value: Union[str, float]) -> float:
    try:
        return float(value)
    except ValueError:
        if isinstance(value, str) and "-" in value:
            parts = value.split("-")
            if len(parts) == 2 and parts[0] and parts[1]:
                try:
                    return (float(parts[0]) + float(parts[1])) / 2
                except ValueError:
                    pass
        return np.nan

In [70]:
y = df['Processed tmp/ºC'].apply(convert_to_float)
invalid_y_ind = y.isna()
invalid_y = y[invalid_y_ind]
invalid_y_ind.sum(), invalid_y

(0, Series([], Name: Processed tmp/ºC, dtype: float64))

In [42]:
invalid_y_ind = y[invalid_y].index
invalid_y_ind

Index([], dtype='int64', name='INDEX')

In [60]:
np.append(invalid_smiles.index.values, invalid_y.index.values)


array([  53,   54,   55,   86,  112,  167,  168,  169,  232,  236,  290,
        336,  362,  363,  364,  366,  393,  402,  408,  418,  421,  449,
        479,  483,  502,  505,  534,  620,  656,  669,  671,  676,  707,
        713,  735,  747,  781,  786,  792,  938, 1028, 1074, 1113, 1129,
       1176, 1263, 1312, 1332, 1333, 1348, 1365, 1366, 1432, 1452, 1464,
       1468, 1492, 1497, 1499, 1565, 1703, 1727, 1731, 1755, 1759, 1777,
       1783, 1814, 1818, 1963, 1967, 1968, 1992, 2028, 2031, 2047, 2048,
       2078, 2110, 2182, 2410, 2417, 2444, 2447, 2460, 2494, 2532, 2610,
       2783, 2803, 2814, 2926, 3200, 3243, 3244, 3253, 3305, 3306, 3313,
       3315, 3319, 3329, 3346, 3348, 3370, 3376, 3380, 3414, 3417, 3467,
       3540, 3579, 3644, 3799, 3817, 3853, 3857, 3875, 3894, 3895, 3900,
       3917, 3920, 3921, 3922, 3929, 4049, 4050, 4066, 4128, 4150, 4152,
       4278, 4376, 4398, 4407, 4418, 4427, 4493, 4551, 4558, 4622, 4650,
       4672, 4675, 4685, 4708, 4710, 4728, 4731, 47